1. Import libraries
2. Manually create training data (No ImageDataGenerator or tf.data used here, for practice)
3. Initialize Keras model 
4. Initialize callbacks
5. Compile, fit and validate model
6. Visualize model training parameters
7. Manually create test data
8. Predict labels
9. Create submission file

## 1. Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Conv2D, MaxPooling2D
import zipfile
import cv2
# from tensorflow.python.keras

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
filenames = os.listdir('../input/dogs-vs-cats')
print(filenames)

In [ ]:
with zipfile.ZipFile("../input/dogs-vs-cats/train.zip","r") as z:
    z.extractall(".")
with zipfile.ZipFile("../input/dogs-vs-cats/test1.zip","r") as z:
    z.extractall(".")
print('Images extracted!')

In [ ]:
# The train and test directories are extracted in the current working directory
print(os.listdir("/kaggle/working/train")[:10])

In [ ]:
base_dir = "/kaggle/working/"
train_dir = "train"
path = os.path.join(base_dir,train_dir)

for p in os.listdir(path):
    category = p.split(".")[0]
    img_array = cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)
    new_img_array = cv2.resize(img_array, dsize=(80, 80))
    plt.imshow(new_img_array,cmap="gray")
    break

## 2. Manually create training data

In [ ]:
X = []
y = []
convert = lambda category : int(category == 'dog')
def create_test_data(path):
    for p in os.listdir(path):
        category = p.split(".")[0]
        category = convert(category)
        img_array = cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)
        new_img_array = cv2.resize(img_array, dsize=(80, 80))
        X.append(new_img_array)
        y.append(category)

In [ ]:
# Reshaping Image array to (batch_size, img height, img width, no. of channels) 
# as required by conv2d (no batches used here, so we write -1 for python to figure out the correct size dimension)
create_test_data(path)
X = np.array(X).reshape(-1, 80,80,1)
y = np.array(y)

In [ ]:
# Find number of instances of both classes in training data
print(X.shape)
print(y.shape)
df = pd.DataFrame({'class':y})
df.value_counts().plot.bar()

In [ ]:
#Normalize data
X = X/255.0

In [ ]:
# print(X)
# print(y)

## 3. Initialize model

In [ ]:
def initialize_model():
    input = Input(shape=(80,80,1))
    x = Conv2D(64,(3,3),activation='relu')(input)
    x = MaxPooling2D(pool_size = (2,2))(x)
    x = Conv2D(64,(3,3),activation='relu')(input)
    x = MaxPooling2D(pool_size = (2,2))(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(1, activation='sigmoid')(x)
    
    return Model(inputs=input, outputs=x)

In [ ]:
model = initialize_model()
model.summary()

## 4. Initialize callbacks

In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

earlystop = EarlyStopping(monitor='val_loss', patience=5)
learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
callbacks = [earlystop, learning_rate_reduction]

## 5. Compile, fit and validate model

In [ ]:
model.compile(optimizer="adam",
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(X, y, epochs=10, batch_size=32, validation_split=0.2, callbacks=callbacks)

## 6. Visualize model training parameters

In [ ]:
print(history.history.keys())

In [ ]:
# Summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['acc','val_acc'], loc='upper left')
plt.show()

In [ ]:
# Summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['loss','val_loss'], loc='upper left')
plt.show()

## 7. Manually create test data

In [ ]:
test_dir = "test1"
path = os.path.join(base_dir,test_dir)

X_test = []
id_line = []

def create_test1_data(path):
    for p in os.listdir(path):
        id_line.append(p.split(".")[0])
        img_array = cv2.imread(os.path.join(path,p),cv2.IMREAD_GRAYSCALE)
        new_img_array = cv2.resize(img_array, dsize=(80, 80))
        X_test.append(new_img_array)
        
create_test1_data(path)
X_test = np.array(X_test).reshape(-1,80,80,1)
X_test = X_test/255

## 8. Predict labels

In [ ]:
predictions = model.predict(X_test)
predictions

In [ ]:
# Rounding predictions to 0 or 1
predicted_val = [int(round(p[0])) for p in predictions]
print(predicted_val[:10])

## 9. Create submission file

In [ ]:
# Create submission file
submission_df = pd.DataFrame({'id':id_line, 'label':predicted_val})
submission_df.to_csv("submission.csv", index=False)